# SSMR: generate SSM model from trajectory data

In [1]:
import numpy as np
from scipy.io import loadmat

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import plot_utils as plot
import matplotlib.pyplot as plt
import utils as utils


In [3]:
use_data = "john" # "john"

In [ ]:
import os
import pickle

In [ ]:
diamond_dir = "../../../soft-robot-control/examples/diamond"
decayData_dir = os.path.join(diamond_dir, "dataCollection/")

OUTPUT_NODE = 1354
t_in = 1.2
t_out = 3.0

oData = []

with open(os.path.join(diamond_dir, 'rest.pkl'), 'rb') as file:
    q_rest = np.array(pickle.load(file)['rest'])

print(q_rest)

decayData_files = sorted([f for f in os.listdir(decayData_dir) if '.pkl' in f]) # [0:1]
plt.close('all')

for i, traj in enumerate(decayData_files):
    with open(os.path.join(decayData_dir, traj), 'rb') as file:
        data = pickle.load(file)

    t = np.array(data['t'])
    q_tip = (np.array(data['q'])[:, 3*OUTPUT_NODE:3*OUTPUT_NODE+3] - q_rest[3*OUTPUT_NODE:3*OUTPUT_NODE+3]).T
    q_dot_tip = np.gradient(q_tip, axis=1)

    # remove time until t_in
    ind = (t_in <= t) & (t <= t_out)
    t = t[ind] - t_in
    q_tip = q_tip[:, ind]
    q_dot_tip = q_dot_tip[:, ind]

    oData_traj = np.vstack([q_tip, q_dot_tip])

    oData.append([t, oData_traj])

In [ ]:
oData[0][1].shape

## Import and inspect data
See mat file in URL

In [4]:
dataDecayObs = loadmat("../dataDecayObs.mat") # , simplify_cells=True)


Data = {}
if use_data == "john":
    Data['oData'] = dataDecayObs['oData'].tolist()
    Data['yData'] = dataDecayObs['yData'].tolist()
    Xsnapshots = dataDecayObs['Xsnapshots']
    for i in range(len(Data['oData'])):
        Data['oData'][i][0] = Data['oData'][i][0].squeeze()
        Data['yData'][i][0] = Data['yData'][i][0].squeeze()
else: # use my data
    Data['oData'] = oData

H = np.array(dataDecayObs['H'])
V = np.array(dataDecayObs['V'])
M = np.array(dataDecayObs['M'])

# oData = dataDecayObs['oData']
# yData = dataDecayObs['yData']

rDOF = 3
oDOF = 3
nTRAJ = len(Data['oData'])
nDOF = len(Xsnapshots)//2

print("nTRAJ:", nTRAJ)
print(H.shape, V.shape, M.shape)

nTRAJ: 52
(4884, 4) (9768, 6) (4884, 4884)


In [ ]:
Xsnapshots.shape

In [ ]:
samples = 0
for i in range(nTRAJ):
    samples += Data['oData'][i][1].shape[1]
    # print(Data['oData'][i][1].shape[1])
print(samples)

In [ ]:
print(V[:nDOF, :rDOF].shape)
print(Data['oData'][0][1][:3, :].shape)
test_y = V[:nDOF, :rDOF].T @ Data['oData'][0][1][:3, :]
print(test_y.shape)
print(np.allclose(test_y, Data['yData'][0][1]))

### Plot trajectories in an observable of interest

In [5]:
outdofs = [0, 1, 2]
plt.close('all')
# plot trajectories in 3D [x, y, z] space
plot.traj_3D(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'])
# plot evolution of x, y and z in time, separately in 3 subplots
plot.traj_xyz(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'])
# plot trajectories in 3D [x, x_dot, z] space / NB: x_dot = v_x
plot.traj_3D(Data,
             xyz_idx=[('oData', 0), ('oData', 0+oDOF), ('oData', 2)],
             xyz_names=[r'$x$', r'$\dot{x}$', r'$z$'])

### Show PCA infos
Modes were already computed to get yData

In [ ]:
oData[0][1].shape

Compute reduced coordinates using delay embedding on the available oData

In [6]:
# Xsnapshots = np.zeros((15, nTRAJ*3000))
delay_embedded_oData = utils.delayEmbedding(Data['oData'], up_to_delay=4)
print(len(delay_embedded_oData))
Xsnapshots = np.hstack([oData[1] for oData in delay_embedded_oData])
print(Xsnapshots.shape)

Data['oData'] = delay_embedded_oData

TypeError: list indices must be integers or slices, not tuple

In [7]:
# Perform SVD on displacement field
# if use_data == "mine":
from scipy.sparse.linalg import svds

# Xsnapshots = np.hstack([oData_traj[1][:, :] for oData_traj in Data['oData']])
# else:
    # compute_uv = False
# s = np.linalg.svd(Xsnapshots[0:nDOF, :], compute_uv=False)
v, s, _ = svds(Xsnapshots, k=6, which="LM")
ind = np.argsort(s)[::-1]
s = s[ind]
v = v[:, ind]
# v, s, _ = np.linalg.svd(Xsnapshots, compute_uv=True)
print(s)
print(v)
l2vals = s**2
len(l2vals)

[10869.94869342  7618.54613271  6721.91306322  1052.58299256
   756.67591794   593.53431604]
[[ 1.44855228e-01  1.90729072e-01 -3.75664223e-01 -2.59277754e-01
   3.27386535e-01  4.31010560e-01]
 [-9.41777671e-02  4.01846948e-01  1.67795833e-01  1.89858273e-01
   5.05943253e-01 -2.64936856e-01]
 [ 4.10759347e-01  2.48130847e-02  1.71558212e-01 -5.09685699e-01
   2.20865683e-02 -3.28987545e-01]
 [ 1.45795055e-01  1.92024607e-01 -3.77675613e-01 -1.58728324e-01
   2.00168942e-01  2.67661344e-01]
 [-9.49353274e-02  4.04063521e-01  1.68769066e-01  1.16163040e-01
   3.09991785e-01 -1.64377142e-01]
 [ 4.12845990e-01  2.49181015e-02  1.71819180e-01 -3.13434103e-01
   1.36501594e-02 -1.92550755e-01]
 [ 1.46090860e-01  1.92565973e-01 -3.78294511e-01  3.57215386e-04
  -3.88646203e-04  5.55478980e-03]
 [-9.52480363e-02  4.04746469e-01  1.69180567e-01 -5.46691521e-04
   2.44031854e-04 -3.17069423e-03]
 [ 4.13536171e-01  2.50411995e-02  1.71936539e-01 -3.60890041e-04
   9.51265272e-05  1.01704047e-02

6

In [8]:
v.shape

(15, 6)

In [9]:
# Plot variance description: we expect three modes to capture almost all variance.
# Note we assume data centered around the origin, which is the fixed point of our system.
plt.close('all')
plot.pca_modes(l2vals, up_to_mode=9)

## Project onto {SSMDim} dominant modes to obtain yData

In [10]:
from copy import deepcopy
yData = deepcopy(Data['oData'])
SSMDim = 6
Vde = v[:, :SSMDim]
print(Data['oData'][0][1].shape, Vde.shape)
for i in range(nTRAJ):
    yData[i][1] = Vde.T @ Data['oData'][i][1]
print(yData[0][1].shape)

(15, 3000) (15, 6)
(6, 3000)


In [ ]:
# if use_data == "mine":
Data['yData'] = yData

## Obtain reduced-order coordinates

In [11]:
plt.close('all')
# plot trajectories in 3D [x1_bar, x1_bar_dot, y] space
plot.traj_3D(Data,
             xyz_idx=[('yData', 0), ('yData', 0+rDOF), ('oData', 1)],
             xyz_names=[r'$\bar{x}_1$', r'$\dot{\bar{x}}_1$', r'$y$'])
# plot evolution of x, y and z in time, separately in 3 subplots
plot.traj_xyz(Data,
              xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
              xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'])

### Train and test data (train/test split)

In [12]:
if use_data == "john":
    indTest = [0, 2, 4, 6, 8, 10, 12, 14]
    indTrain = [i for i in range(nTRAJ) if i not in indTest]
else:
    indTest = [0, 2, 4, 6, 8]
    indTrain = [i for i in range(nTRAJ) if i not in indTest]

### Truncate trajectories

In [13]:
t_interval = [0.14, 2.6]
Data['oDataTrunc'] = utils.slice_trajectories(Data['oData'], t_interval)
Data['yDataTrunc'] = utils.slice_trajectories(Data['yData'], t_interval)

In [ ]:
print(Data['oDataTrunc'][0][0].shape)
print(Data['oDataTrunc'][0][1].shape)

Visualize truncated trajectories

In [14]:
plt.close('all')
plot.traj_3D(Data,
             xyz_idx=[('yDataTrunc', 0), ('yDataTrunc', 0+rDOF), ('oDataTrunc', 1)],
             xyz_names=[r'$\bar{x}_1$', r'$\dot{\bar{x}}_1$', r'$y$'])

In [15]:
plt.close('all')

## Use Matlab engine to evoke SSMLearn

Start Matlab engine and install/run SSMLearn

In [16]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.cd("../..", nargout=0)
eng.install(nargout=0)
eng.cd("ROM/")

SSMLearn: Data-driven Reduced Order Models for Nonlinear Dynamical Systems
Maintained by Mattia Cenedese (mattiac@ethz.ch) and Joar Axs (jgoeransson@ethz.ch)
License: GNUv3.0
> In path (line 109)
In addpath (line 86)
In install (line 20)


'/home/jonas/Projects/stanford/SSMR-for-control'

Bring data in a format that is accepted by the (slightly modified) version of SSMLearn (requires a specific cell array structure for input data)

In [17]:
# make data ready for Matlab
oDataTruncTrain_matlab = [[], []]
yDataTruncTrain_matlab = [[], []]
for i in indTrain:
    oDataTruncTrain_matlab[0].append(matlab.double(initializer=Data['oDataTrunc'][i][0].tolist()))
    oDataTruncTrain_matlab[1].append(matlab.double(initializer=Data['oDataTrunc'][i][1].tolist()))
    yDataTruncTrain_matlab[0].append(matlab.double(initializer=Data['yDataTrunc'][i][0].tolist()))
    yDataTruncTrain_matlab[1].append(matlab.double(initializer=Data['yDataTrunc'][i][1].tolist()))


### Learn geometry of the SSM

Find parametrization of SSM using SSMLearn

In [18]:
SSMDim = 2*rDOF
obsDim = 2*oDOF
SSMOrder = 3

IMInfo = eng.IMGeometry(oDataTruncTrain_matlab, SSMDim, SSMOrder,
                        'reducedCoordinates', yDataTruncTrain_matlab)
IMInfoInv = eng.IMGeometry(yDataTruncTrain_matlab, obsDim, SSMOrder,
                           'reducedCoordinates', oDataTruncTrain_matlab)


Data seems to have been fed from Python!
Data seems to have been fed from Python!
> In ridgeRegression (line 28)
In IMGeometry (line 109)



In [ ]:
IMInfo

### Learn dynamics on the SSM (reduced dynamics)

Find parametrization of reduced dynamics using SSMLearn

In [19]:
ROMOrder = 3
RDInfo = eng.IMDynamicsFlow(yDataTruncTrain_matlab, 'R_PolyOrd', ROMOrder, 'style', 'default')
# RDInfo['eigenvaluesLinPartFlow'][0:3]
RDInfo['conjugateDynamics']['polynomialOrder']

Data seems to have been fed from Python!
Estimation of the reduced dynamics... 
 Done. 


3.0

Stop Matlab engine -- not needed anymore

In [20]:
eng.quit()

## Analyze the obtained mappings of SSM geometry and reduced dynamics

### SSM geometry (parametrization)

In [21]:
oRec = {}
errorGeo = {}
meanErrorGeo = {}

#### Train error

In [22]:
oRec['Train'] = utils.lift_trajectories(IMInfo, [Data['yDataTrunc'][i] for i in indTrain])
errorGeo['Train'] = utils.compute_trajectory_errors(oRec['Train'], [Data['oDataTrunc'][i] for i in indTrain])[0] * 100

meanErrorGeo['Train'] = np.mean(errorGeo['Train'])
print(f"Average parametrization train error: {meanErrorGeo['Train']:.4f}")

Average parametrization train error: 0.0725


#### Test error

In [23]:
oRec['Test'] = utils.lift_trajectories(IMInfo, [Data['yDataTrunc'][i] for i in indTest])
errorGeo['Test'] = utils.compute_trajectory_errors(oRec['Test'], [Data['oDataTrunc'][i] for i in indTest])[0] * 100

meanErrorGeo['Test'] = np.mean(errorGeo['Test'])
print(f"Average parametrization test error: {meanErrorGeo['Test']:.4f}")

Average parametrization test error: 0.0716


Plot comparison of SSM-predicted vs. actual test trajectories

In [24]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(oRec,
              xyz_idx=[('Test', 0), ('Test', 1), ('Test', 2)],
              xyz_names=[r'$x$', r'$y$', r'$z$'],
              axs=axs, ls=':', color='darkblue')

### SSM geometry (chart)

In [25]:
yRec = {}

In [30]:
IMInfoInv
np.array(IMInfoInv['parametrization']['H']).shape

(6, 83)

In [31]:
IMInfo
np.array(IMInfo['parametrization']['H']).shape

(15, 83)

#### Train error

In [26]:
yRec['CTrain'] = utils.lift_trajectories(IMInfoInv, [Data['oDataTrunc'][i] for i in indTrain])
errorGeo['CTrain'] = utils.compute_trajectory_errors(yRec['CTrain'], [Data['yDataTrunc'][i] for i in indTrain])[0] * 100

meanErrorGeo['CTrain'] = np.mean(errorGeo['CTrain'])
print(f"Average chart train error: {meanErrorGeo['CTrain']:.4f}")

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 815 is different from 83)

#### Test error

In [ ]:
yRec['CTest'] = utils.lift_trajectories(IMInfoInv, [Data['oDataTrunc'][i] for i in indTest])
errorGeo['CTest'] = utils.compute_trajectory_errors(yRec['CTest'], [Data['yDataTrunc'][i] for i in indTest])[0] * 100

meanErrorGeo['CTest'] = np.mean(errorGeo['CTest'])
print(f"Average chart test error: {meanErrorGeo['CTest']:.4f}")

Plot comparison of SSM-predicted vs. actual test trajectories

In [ ]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(yRec,
              xyz_idx=[('CTest', 0), ('CTest', 1), ('CTest', 2)],
              xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
              axs=axs, ls=':', color='darkblue')

Store map in the same variables

In [ ]:
for key in ['map', 'polynomialOrder', 'dimension', 'nonlinearCoefficients', 'phi', 'exponents', 'H']:
    IMInfo['chart'][key] = IMInfoInv['parametrization'][key]

### Reduced dynamics

In [32]:
yRec['RD'] = utils.advectRD(RDInfo, Data['yDataTrunc'])[0]
meanErrorDyn = {}

normedTrajDist = utils.compute_trajectory_errors(yRec['RD'], Data['yDataTrunc'])[0]
meanErrorDyn['Train'] = np.mean(normedTrajDist[indTrain]) * 100
meanErrorDyn['Test'] = np.mean(normedTrajDist[indTest]) * 100

print(f"Average dynamics train error: {meanErrorDyn['Train']:.4f}")
print(f"Average dynamics test error: {meanErrorDyn['Test']:.4f}")

Average dynamics train error: 1.8632
Average dynamics test error: 0.7939


In [33]:
plot_all = False
if plot_all:
    vPlot = range(nTRAJ)
else:
    vPlot = indTest

plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
                    traj_idx=vPlot,
                    show=False)
plot.traj_xyz(yRec,
              xyz_idx=[('RD', 0), ('RD', 1), ('RD', 2)],
              xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
              traj_idx=vPlot,
              axs=axs, ls=':', color='darkblue')


### Global error

In [34]:
errorGlo = {}
meanErrorGlo = {}

#### Train error

In [35]:
oRec['RDTrain'] = utils.lift_trajectories(IMInfo, [yRec['RD'][i] for i in indTrain])
errorGlo['Train'] = utils.compute_trajectory_errors(oRec['RDTrain'], [Data['oDataTrunc'][i] for i in indTrain])[0] * 100

meanErrorGlo['Train'] = np.mean(errorGlo['Train'])
print(f"Average global train error: {meanErrorGlo['Train']:.4f}")

Average global train error: 1.0815


#### Test error

In [36]:
oRec['RDTest'] = utils.lift_trajectories(IMInfo, [yRec['RD'][i] for i in indTest])
errorGlo['Test'] = utils.compute_trajectory_errors(oRec['RDTest'], [Data['oDataTrunc'][i] for i in indTest])[0] * 100

meanErrorGlo['Test'] = np.mean(errorGlo['Test'])
print(f"Average global test error: {meanErrorGlo['Test']:.4f}")

Average global test error: 0.7062


In [37]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(oRec,
              xyz_idx=[('RDTest', 0), ('RDTest', 1), ('RDTest', 2)],
              xyz_names=[r'$x$', r'$y$', r'$z$'],
              axs=axs, ls=':', color='darkblue')

In [ ]:
RDInfo['reducedDynamics']['coefficients']

### Understand which mode is responsible for what motion

In [38]:
outdofsMat = np.zeros((3, 2*nDOF))
output_node = 1354
outdofsPS = 3 * output_node + np.array([0, 1, 2])
for i in range(3):
    outdofsMat[i, outdofsPS[i]] = 1

redDynLinearPart = np.array(RDInfo['reducedDynamics']['coefficients'])[:, :2*rDOF]
redStiffnessMat = -redDynLinearPart[rDOF:, :rDOF]

w2, UconsRedDyn = np.linalg.eig(redStiffnessMat)
sorted_idx = np.argsort(w2)
print(w2)
consFreqs, UconsRedDyn = np.diag(np.sqrt(w2[sorted_idx])), UconsRedDyn[:, sorted_idx]

Ucons = np.kron(np.eye(2), UconsRedDyn)
Vcons = V @ Ucons

modesDir = outdofsMat @ Vcons[:, :rDOF]
modesFreq = np.abs(RDInfo['eigenvaluesLinPartFlow']).flatten()[:3]

plt.close('all')
plot.mode_direction(modesDir, modesFreq)
modesDir

[351.48229067 293.93793445 306.45403062]


array([[ 0.03813484,  0.01939068, -0.01825275],
       [ 0.01420079, -0.04024069, -0.00616473],
       [ 0.02167104,  0.00492282,  0.04446898]])

## Control

$H_{red} = V^TM^{-1}H$

In [39]:
Hred = V[:nDOF, :rDOF].T @ np.linalg.solve(M, H)

normsModalForcing = np.mean(Hred**2, axis=0)
normsNonModalforcing = np.mean(((np.eye(nDOF) - V[:nDOF, :rDOF] @ V[:nDOF,:rDOF].T) @ np.linalg.solve(M, H)) ** 2, axis=0)

In [40]:
normsModalForcing

array([3616.45209364, 3685.11835952, 2706.933674  , 3629.53852193])

In [41]:
normsNonModalforcing

array([1874.58976771, 1541.09869231, 1333.69786275, 1201.55627627])

### Setup model for control

In [42]:
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d 

In [43]:
np.array(IMInfo['parametrization']['H']).shape
np.array(IMInfo['chart']['H']).shape

KeyError: 'H'

In [137]:
Rauton = utils.Rauton(RDInfo)
C = lambda x: IMInfo['parametrization']['H'] @ utils.multivariate_polynomial(x, IMInfo['parametrization']['polynomialOrder'])
W = lambda x: IMInfo['chart']['H'] @ utils.multivariate_polynomial(x, IMInfo['chart']['polynomialOrder'])

Small example

In [138]:
uFun = lambda t: np.array([np.arctan(t)/np.pi*2, 0, 0, 0]).reshape(-1, 1)
def R(t, y):
    print(np.linalg.norm(y))
    return Rauton(np.atleast_2d(y)) + np.vstack([np.zeros((rDOF, Hred.shape[1])), Hred]) @ uFun(t)

tTraj = np.linspace(0, 15, 1000)
sol = solve_ivp(R,
                t_span=[tTraj[0], tTraj[-1]],
                t_eval=tTraj,
                y0=np.zeros(2*rDOF),
                method='LSODA',
                vectorized=True,
                rtol=1e-3,
                atol=1e-3)
yTraj = sol.y
oTraj = C(yTraj)

print(oTraj.shape)

plt.close('all')
plot.traj_xyz_txyz(tTraj,
                   oTraj[outdofs[0], :],
                   oTraj[outdofs[1], :],
                   oTraj[outdofs[2], :])

0.0
0.0
13.931243309570654
0.0
0.928156452349539
0.192747031483198
0.0
0.05826362142622365
0.046217503304057106
0.0
0.0036424762332300715
0.0
0.00190233904013724
0.0036622375966743844
0.005464227569526993
0.015918025934738605
0.015714085715573332
0.0324249186367721
0.03201697954799346
0.05377756968793556
0.053283243985518224
0.12455360318835618
0.12259894110640614
0.19721986745595396
0.19770066426089658
0.2636432560466361
0.26418917798220476
0.30582384170935983
0.3081438781758471
0.31194076392185555
0.3119357567444737
0.31483570174787645
0.31485302199543547
0.31738867132712967
0.3173883386145724
0.31966702040790246
0.31966740025324736
0.32151562814241236
0.3215167280404704
0.3229384989594017
0.32294032974529147
0.32387777458541506
0.32387984279724114
0.32446531040258575
0.32446780754231463
0.3247076000209288
0.32471056649600955
0.32374538503380884
0.323786804150253
0.32067406783858926
0.32074600300633754
0.31568313064362186
0.31577185031202054
0.3028134739271701
0.30323662528858625
0.2

### Integrate the model with inputs

#### Small inputs

In [175]:
# Load the controls and actual trajectories for figure8
uFigureEight = np.loadtxt("../../system_data/u_small.csv", delimiter=',')
z = np.loadtxt("../../system_data/z_small.csv", delimiter=',')
z_eq = np.loadtxt("../../system_data/z_equilibrium.csv", delimiter=',')
z_eq = np.concatenate([z_eq, np.zeros(3*4)])

# original spacing of inputs
T = 10.01
dt = 0.01
N = int(T / dt)
t_span = np.linspace(0, T, N+1)
z_true = z[:, 3:].T

interpFun = interp1d(t_span, uFigureEight, axis=0, fill_value="extrapolate")
uFun = lambda t: interpFun(t).reshape(-1, 1)
R = lambda t, y: Rauton(np.atleast_2d(y)) + np.vstack([np.zeros((rDOF, Hred.shape[1])), Hred]) @ uFun(t)

sol = solve_ivp(R,
                t_span=[t_span[0], t_span[-1]],
                t_eval=t_span,
                y0=np.zeros(2*rDOF),
                method='LSODA',
                vectorized=True,
                rtol=1e-3,
                atol=1e-3)
yTraj = sol.y
oTraj = (C(yTraj).T + z_eq).T

plt.close('all')
ax = plot.traj_3D_xyz(oTraj[outdofs[0], :],
                      oTraj[outdofs[1], :],
                      oTraj[outdofs[2], :], show=False)
plot.traj_3D_xyz(z_true[outdofs[0], :],
                 z_true[outdofs[1], :],
                 z_true[outdofs[2], :], color="tab:orange", ax=ax)


#### Big inputs

In [222]:
# Load the controls and actual trajectories for figure8
uFigureEight = np.loadtxt("../../system_data/u_big.csv", delimiter=',')
z = np.loadtxt("../../system_data/z_big.csv", delimiter=',')
z_eq = np.loadtxt("../../system_data/z_equilibrium.csv", delimiter=',')
z_eq = np.concatenate([z_eq, np.zeros(3)])
print(z_eq)

[-7.07686434  1.2132194  97.04053866  0.          0.          0.        ]


In [223]:
plt.close('all')

### Learn control matrix $B$

Use figure8 trajectory to identify the impact of control, i.e. regress the control matrix $B$

In [224]:
diamond_train_full = loadmat("../diamond_train_full.mat", simplify_cells=True)
y, u, t = diamond_train_full['y'], diamond_train_full['u'], diamond_train_full['t']

print(y.shape)

y = y.T
U = u.T

z_true_train = np.vstack([y[3:, :], y[:3, :]])
print(z_true_train.shape)
z_shift_train = (z_true_train.T - z_eq).T
z_shift_train = utils.delayEmbedding(z_shift_train)

# z_shift_train = z_true_train
t_span_train = np.hstack([t, t_span[1:] + t[-1]])

# # Add test trajectory to training dataset
# U = np.hstack([U, U_eight[:, 1:]])
# z_shift = np.hstack([z_shift, z_eight_shift[:, 1:]])

# Use only test trajectory as the training dataset --> should well overfit
# U = U_eight[:, 1:]
# z_shift_train = z_eight_shift[:, 1:]

# Xbar = np.array(IMInfo['chart']['H']) @ utils.multivariate_polynomial(z_shift_train, IMInfo['chart']['polynomialOrder'])
print(Vde.shape)
Xbar = Vde.T @ z_shift_train
dXbarDt = np.gradient(Xbar, dt, axis=1)[:, 10:-10]

print(U.shape)
U = U[:, 10:-10]
Xbar = Xbar[:, 10:-10]

dXbarDt_ROM = Rauton(Xbar)

(8692, 6)
(6, 8692)
(15, 6)
(4, 8692)


In [225]:
plt.close('all')
ax = plot.traj_3D_xyz(z_true_train[outdofs[0], :],
                      z_true_train[outdofs[1], :],
                      z_true_train[outdofs[2], :], show=True)

Learn $B$ matrix

In [226]:
from sklearn.linear_model import Ridge

polyUorder = 1

# X = np.vstack([U, Xbar])
X = utils.multivariate_polynomial(U, order=polyUorder)
print(X.shape)

ridgeModel = Ridge(alpha=0, fit_intercept=False)
# ridgeModel.fit(U.T, (dXbarDt - dXbarDt_ROM).T)
ridgeModel.fit(X.T, (dXbarDt - dXbarDt_ROM).T)
B_learn = ridgeModel.coef_
print(B_learn.shape)

(4, 8672)
(6, 4)


In [181]:
# # home-brewed ridge regression
# l = 0
# X, y = U.T, (dXbarDt - dXbarDt_ROM).T
# B_learn = np.linalg.solve(X.T @ X + l * np.eye(X.shape[0]), X.T @ y)

### Integrate moel with inputs (big amplitude) -- and using learned influence of control

In [228]:
# original spacing of inputs
T = 10.01
dt = 0.01
N = int(T / dt)
t_span = np.linspace(0, T, N+1)
z_true = z[:, 3:].T


interpFun = interp1d(t_span, uFigureEight, axis=0, fill_value="extrapolate")
uFun = lambda t: interpFun(t).reshape(-1, 1)
def R(t, y):
    # print(y.shape)
    return Rauton(np.atleast_2d(y)) + B_learn @ utils.multivariate_polynomial(uFun(t), order=polyUorder) #  np.vstack([np.zeros((rDOF, Hred.shape[1])), Hred]) @ uFun(t) # np.vstack([uFun(t), y])

sol = solve_ivp(R,
                t_span=[t_span[0], t_span[-1]],
                t_eval=t_span,
                y0=np.zeros(2*rDOF),
                method='RK45',
                vectorized=True,
                rtol=1e-3,
                atol=1e-3)
yTraj = sol.y
# oTraj = (C(yTraj).T + z_eq).T
oTraj = C(yTraj)

plt.close('all')
ax = plot.traj_3D_xyz(oTraj[outdofs[0], :],
                      oTraj[outdofs[1], :],
                      oTraj[outdofs[2], :], show=False)
ax = plot.traj_3D_xyz(z_true[outdofs[0], :],
                      z_true[outdofs[1], :],
                      z_true[outdofs[2], :], color="tab:orange", ax=ax, show=False)
ax.legend(["Predicted trajectory", "Actual trajectory"])
plt.show()


In [210]:
RMSE = np.sum(np.sqrt(np.mean((oTraj[0:3, :] - z_true)**2, axis=0))) / oTraj.shape[1]
print(f"RMSE = {RMSE:.4f}")

RMSE = 68.3634
